# Create DA Bid Function Plots
---

In [ ]:
import sys, os
import re
print(os.getcwd())
sys.path.append(os.path.realpath('.'))
sys.path.append(os.path.join(os.path.expanduser('~'), 'bnelearn'))
sys.path.append(os.path.join(os.path.expanduser('~'), 'projects/bnelearn'))
import torch
import matplotlib.pyplot as plt
import numpy as np
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
markers = ['v', '^']

from bnelearn.experiment.configuration_manager import ConfigurationManager
from bnelearn.strategy import NeuralNetStrategy
from bnelearn.experiment.equilibria import (
    bne_bilateral_bargaining_uniform_linear,
    bne_bilateral_bargaining_uniform_symmetric)

### A. Load models

In [ ]:
dir_path = "/home/pieroth/projects/bnelearn/experiments/bargaining_paper_results/exp-3_experiment/double_auction/single_item/k_price/0.5/uniform/symmetric/risk_1.0/1b1s/2021-10-23 Sat 00.02/00 00:46:38 0/models"

models = list()
for file in os.listdir(dir_path):
    if file.endswith(".pt"):
        model_path = dir_path + '/' + file
        models.append(NeuralNetStrategy.load(model_path))
print(f'Found and loaded {len(models)} models in path.')

In [ ]:
model_names = ['buyer', 'seller']

### B. Load BNEs

In [ ]:
experiment_config = \
    ConfigurationManager(
        experiment_type='double_auction_single_item_uniform_symmetric',
        n_runs=1,  # repeat exp. for different random seeds
        n_epochs=0,
    )
experiment_config.setting.k = 0.5
experiment_config.setting.n_buyers = 1
bne_strategies = [bne_bilateral_bargaining_uniform_linear(experiment_config, 0, 1)]
bne_strategies += bne_bilateral_bargaining_uniform_symmetric(experiment_config)


In [ ]:
bne_names = ['linear', '0.25', '0.45']

### C. Plot

In [ ]:
v = torch.linspace(0, 1, 35).view(-1, 1)

for i, model in enumerate(models):
    learned_bid = model.play(v).detach()
    plt.plot(
        v.numpy(), learned_bid.numpy(),
        markers[i], markersize=7,
        label=f'NPGA {model_names[i]}',
        color=colors[i])
    for j, bne_strategy in enumerate(bne_strategies):
        bne_bid = bne_strategy(v, i)
        plt.plot(
            v.numpy(), bne_bid.numpy(), '--',
            label=f'BNE {model_names[i]} {bne_names[j]}',
            color=colors[i+j])
plt.xlabel('valuation/cost $v$'); plt.ylabel('bid $b(v)$')
plt.xlim([0, 1]); plt.ylim([0, 1])
plt.legend(); plt.tight_layout()
plt.savefig('test.eps'); plt.show()

In [ ]:
# Plot symmetric equilibrium strategies
all_g_05s = np.linspace(0.04, 0.44, 4)
suboptimal_strategies = bne_bilateral_bargaining_uniform_symmetric(experiment_config, list(all_g_05s))
suboptimal_bne_names = [str(np.round(value, 2)) for value in all_g_05s]
# color_list = np.repeat(np.linspace(0.5, 1.0, len(suboptimal_bne_names))[:, np.newaxis], 2, axis=1)
# color_list = [str(value_of_values) for values in color_list for value_of_values in values]
color_list = [(0/255.,150/255.,196/255.),(248/255.,118/255.,109/255.),
              (150/255.,120/255.,170/255.),(255/255.,215/255.,130/255.),
              (0/255.,150/255.,196/255.),(248/255.,118/255.,109/255.),
              (150/255.,120/255.,170/255.),(255/255.,215/255.,130/255.)
              ]
marker = ["<", "s"]
linestyles = ["-", "--", "-.", ":", (0, (1, 10)), (0, (1, 1)), (0, (5, 10)), (0, (5, 1)), (0, (3, 10, 1, 10)), (0, (3, 1, 1, 1)), (0, (3, 10, 1, 10, 1, 10)), (0, (3, 1, 1, 1, 1, 1))]

In [ ]:
v = torch.linspace(0, 1, 350).view(-1, 1)
figure_plt = plt.figure()
plt.style.use('ggplot')
for i in [0, 1]:
    for j, bne_strategy in enumerate(suboptimal_strategies):
        bne_bid = bne_strategy(v, i)
        plt.plot(
            v.numpy(), bne_bid.numpy(),
            label=f'BNE {model_names[i]} {suboptimal_bne_names[j]}',
            color=color_list[4*i + j], marker=marker[i], markevery=50,
            linestyle=linestyles[j])
plt.xlabel('Valuation'); plt.ylabel('Bid')
plt.xlim([0, 1]); plt.ylim([0, 1])
plt.legend(); plt.tight_layout()
plt.savefig('sub_optimal_bne_strategies.pdf'); plt.show()